In [6]:
import requests
#import xml.etree.ElementTree as ET
#!pip install lxml
from lxml import etree as ET
import json 
import os 
import shutil



## C'est l'heure du pseudo code  ! 

Parcourir les dossiers de données et récupérer les identifiants de chaque fichier
E.g. : ```1.177. Honours for Zenon son of Zenon, wrestler A.jpg```


Dans le même temps, stocker les chemins(dictionnaire / json ?  )

Une fois les des identirifiants récupérés, parcourir les xml du site pour aller chercher l'image à étudier


Dans le XML, aller chercher l'URL de l'image ``` Xpath : //div[@type='figure' and @n='photographs']//figure/@href```


On le concatène avec l'URL de base du site : ``` https://insaph.kcl.ac.uk/images/photo/full/ + id + .jpg```

On scrape l'image et on la stocke suivante le même nom que le fichier d'origine



Une fois toutes les images récupéréées, on fait un train test split sur toutes les classes, puis on lance un YOLOv11 classifier pour l'entraînement
(Est-ce qu'on garde unsplit 80 20 ou est-ce qu'on ajoute une classe de valisation pour avoir un split 70 20 10 ?)



In [7]:
import os

def get_id(file_name):
    # On récupère l'identifiant de l'image
    elem1, elem2 = file_name.split(".")[:2]
    if len(elem1) +len(elem2)== 6:
        return elem1 + elem2
    if len(elem1) == 1:
        elem1 = '0' + elem1 + '0'
    else:
        elem1 += '0'
    if len(elem2) == 1:
        elem2 = '00' + elem2
    elif len(elem2) == 2:
        elem2 = '0' + elem2
    return elem1 + elem2


In [8]:
def get_img_link(xml_path):
     tree = ET.parse(xml_path)
     root = tree.getroot()
     return  [f'https://insaph.kcl.ac.uk/images/photo/full/{href}.jpg' for href in root.xpath("//div[@type='figure' and @n='photographs']//figure[@href]/@href")]

def get_image(url,path) : 
    img_data = requests.get(url).content
    with open(path, 'wb') as handler:
        handler.write(img_data)
        


In [9]:
import cv2
from sklearn.metrics.pairwise import cosine_similarity
def get_img_sim(img1, img2):
    try:
        # Vérification des chemins
        if not os.path.exists(img1):
            raise FileNotFoundError(f"Fichier introuvable : {img1}")
        if not os.path.exists(img2):
            raise FileNotFoundError(f"Fichier introuvable : {img2}")

        # Chargement des images
        image1 = cv2.imread(img1)
        image2 = cv2.imread(img2)

        if image1 is None:
            raise ValueError(f"Échec du chargement de l'image : {img1}")
        if image2 is None:
            raise ValueError(f"Échec du chargement de l'image : {img2}")

        # Redimensionnement aux dimensions minimales
        min_height = min(image1.shape[0], image2.shape[0])
        min_width = min(image1.shape[1], image2.shape[1])

        if min_height == 0 or min_width == 0:
            raise ValueError("Une des images a une dimension invalide (hauteur ou largeur = 0)")

        image1 = cv2.resize(image1, (min_width, min_height))
        image2 = cv2.resize(image2, (min_width, min_height))

        # Conversion en vecteurs
        vector1 = image1.flatten()
        vector2 = image2.flatten()

        # Calcul de la similarité cosinus
        similarity = cosine_similarity([vector1], [vector2])[0][0]

        return similarity

    except Exception as e:
        print(f"[Erreur] {e}")
        return None

In [10]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# Définir les chemins
fichier_data = "/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data"
fichier_data_clean = "/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data_clean"
xml_dir = "/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/inscriptions"

# Récupération de tous les fichiers .jpg
all_jpg_files = []
for root, dirs, files in os.walk(fichier_data):
    for file in files:
        if file.endswith(".jpg"):
            all_jpg_files.append((root, file))
show = False
# Boucle principale avec tqdm
for root, file in tqdm(all_jpg_files, desc="Traitement des images", total=len(all_jpg_files), leave=True):
    try:
        # On récupère la classe
        classe = root.split("/")[-2] if len(root.split("/")) == 9 else root.split("/")[-1]
        #print(classe)

        # On récupère le nom et l'identifiant de l'image
        nom_image = file.split(".")[0].strip()
        try:
            id_image = get_id(file)
            #print(f"[Info] ID de l'image : {id_image}")
        except ValueError:
            print(f"[Erreur] get_id invalide pour : {file}")
            continue
        if len(id_image) != 6:
            print(f"[Erreur] ID de l'image invalide : {id_image} pour le fichier {file}")
            continue
        # Accès au fichier XML
        xml_path = os.path.join(xml_dir, 'iAph' + id_image + '.xml')
        if not os.path.exists(xml_path):
            print(f"[Erreur] Le fichier XML {xml_path} n'existe pas.")
            continue

        img_links = get_img_link(xml_path)

        # Créer un dossier temporaire
        if not os.path.exists('temp'):
            os.makedirs('temp')

        for ind, img in enumerate(img_links):
            get_image(img, f'temp/{ind}_{id_image}.jpg')

        # Comparaison d'image
        img_files = os.listdir('temp')
        max_similarity = 0
        most_similar_image = None
        original_img_path = os.path.join(root, file)

        for img_file in img_files:
            if img_file.startswith('.'):
                continue
            temp_img_path = os.path.join('temp', img_file)
            similarity = get_img_sim(original_img_path, temp_img_path)
            if similarity is None:
                continue
            if similarity > max_similarity:
                max_similarity = similarity
                most_similar_image = temp_img_path

        # Affichage de la comparaison si souhaité
        if show :  
            if most_similar_image:
                img1 = cv2.imread(original_img_path)
                img2 = cv2.imread(most_similar_image)
                if img1 is not None and img2 is not None:
                    plt.figure(figsize=(8, 4))
                    plt.subplot(1, 2, 1)
                    plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
                    plt.axis("off")
                    plt.subplot(1, 2, 2)
                    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
                    plt.axis("off")
                    plt.tight_layout()
                    plt.show()
                    plt.close()
            else:
                print(f"[Info] Aucune image similaire trouvée pour {file}")

        # Création du dossier propre
        class_dir = os.path.join(fichier_data_clean, classe)
        os.makedirs(class_dir, exist_ok=True)

        # Enregistrement
        target_path = os.path.join(class_dir, file)
        if most_similar_image is not None:
            shutil.copy(most_similar_image, target_path)
        else:
            shutil.copy(original_img_path, target_path)

    finally:
        # Nettoyage
        if os.path.exists('temp'):
            shutil.rmtree('temp', ignore_errors=True)

Traitement des images:   1%|          | 8/1300 [00:10<27:28,  1.28s/it]

[Erreur] Échec du chargement de l'image : temp/2_010403.jpg


Traitement des images:   1%|          | 10/1300 [00:15<35:15,  1.64s/it]

[Erreur] Échec du chargement de l'image : temp/2_010403.jpg


Traitement des images:   1%|          | 14/1300 [00:21<30:08,  1.41s/it]

...

Traitement des images:  93%|█████████▎| 1212/1300 [27:16<01:29,  1.02s/it]

[Erreur] Échec du chargement de l'image : temp/2_110069.jpg
[Erreur] Échec du chargement de l'image : temp/3_110069.jpg
[Erreur] Échec du chargement de l'image : temp/4_110069.jpg


Traitement des images:  96%|█████████▋| 1254/1300 [27:57<00:27,  1.65it/s]

[Erreur] Échec du chargement de l'image : temp/1_150358.jpg


Traitement des images: 100%|██████████| 1300/1300 [28:36<00:00,  1.32s/it]


In [ ]:
import os
all_jpg_files_cleaned = []
for root, dirs, files in os.walk('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data_clean'):
    for file in files:
        if file.endswith(".jpg"):
            all_jpg_files_cleaned.append((root, file))
len(all_jpg_files_cleaned)

1274

In [ ]:
import os
all_jpg_files = []
for root, dirs, files in os.walk('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data'):
    for file in files:
        if file.endswith(".jpg"):
            all_jpg_files.append((root, file))

all_jpg_files

[('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data/Factional',
  '4.13. Acclamations of the Greens, the City and the Blues A.jpg'),
 ('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data/Factional',
  '4.13. Acclamations of the Greens, the City and the Blues C.jpg'),
 ('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data/Factional',
  '2.18. Seat inscriptions, Odeon block D.jpg'),
 ('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data/Factional',
  '4.13. Acclamations of the Greens, the City and the Blues B.jpg'),
 ('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data/Factional',
  '10.4. Seat inscriptions Stadium Block 18.jpg'),
 ('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data/Factional',
  '8.106. Acclamation of city Tyche and the Greens.jpg'),
 ('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data/Factional',
  '8.55. Seat inscriptions Theatr

In [11]:
missing_file = [i for i in all_jpg_files if i not in all_jpg_files_cleaned ]

In [14]:

## move missing files from data to data clean

import os
import shutil

#all_jpg_files = []
for root, dirs, files in os.walk('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data'):
    for file in files : 
        if file in missing_file : 
            classe = root.split("/")[-2] if len(root.split("/")) == 9 else root.split("/")[-1]
            shutil.copy(os.path.join(root, file),os.path.join('/Users/rolly/Documents/10-19_Université_et_scolarité/paleo_comput/data_clean',classe,file))

#all_jpg_files
